# Good hint Migration 
This notebook demonstrates how an LLM performs when presented with a Spring Security migration issue with a good example

Installing pre requisites and configuring Kai with `GPT-3.5-turbo`

In [ ]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

In [30]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
load_dotenv() 

# Initialize the model provider using GPT-4o via OpenAI
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "gpt-3.5-turbo"},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



Below is the snippet we are trying to migrate from springboot2 to springboot3. This code uses the discouraged `configure(AuthenticationManagerBuilder)` method inside a `WebSecurityConfigurerAdapter`, both of which are deprecated in Spring Security 5.7+ and removed in Spring Boot 3.

Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [31]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

This rule flags the use of `configure(AuthenticationManagerBuilder)`. In Spring Boot 3, developers should instead use a `UserDetailsService` and `PasswordEncoder` bean for authentication configuration.

In [32]:
hint_to_migrate="""\
    
  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  message: |
    Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
    Instead, define a `UserDetailsService` and `PasswordEncoder` bean.
    Use a SecurityFilterChain bean to configure HTTP security instead of extending WebSecurityConfigurerAdapter

    """

Similar example showing before and after migration

In [33]:
example_before_code="""\

@Configuration
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        auth.inMemoryAuthentication()
            .withUser(user);
    }
}

"""

example_after_code="""\
@Configuration
public class SecurityConfiguration {
    @Bean
    public InMemoryUserDetailsManager userDetailsService() {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }
}
"""

summary_of_changes="""\
* Removed the WebSecurityConfigurerAdapter class.
* Created a UserDetailsService bean instead of overriding configure(AuthenticationManagerBuilder).
* Defined a SecurityFilterChain bean to handle HTTP security configurations.
* Kept the PasswordEncoder bean for password encoding.

"""

In [34]:
prompt = f"""

The following java code needs to be migrated from springboot2 to springboot3.

```java
{before_code}
```
I will provide you with an example before/after code migration and a summary of changes
```java
{example_before_code}
{example_after_code}
```

{summary_of_changes}

Use the following hint's description and message to migrate. 

{hint_to_migrate}

Also, remove the deprecated and unused APIs and suggest the correct alternatives based on the migration needs. 

Can you provide a refactored version of {before_code} that is compatible with springboot3?
Also, create a step by step process for the above migration? Summarize and create a hint that supplements the data in the prompt

"""

In [35]:

response = await rendered_llm_call(prompt)

Refactored version:

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;
import org.springframework.security.provisioning.InMemoryUserDetailsManager;
import org.springframework.security.core.userdetails.User;
import org.springframework.security.core.userdetails.UserDetails;

@Configuration
@EnableWebSecurity
public class SecurityConfiguration {

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }

    @Bean
    public InMemoryUserDetailsManager userDetailsService() {
        UserDetails user = User.withUsername("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }

    @Bean
    public SecurityFilterChain securityFilterChain(HttpSecurity http) throws Exception {
        http
            .authorizeRequests(authorizeRequests ->
                authorizeRequests
                    .anyRequest().authenticated()
            )
            .formLogin(withDefaults());
        return http.build();
    }
}
```

Step by step process for migration:
1. Remove the `WebSecurityConfigurerAdapter` extension and override of `configure(AuthenticationManagerBuilder)` method.
2. Create a `SecurityConfiguration` class with `@Configuration` and `@EnableWebSecurity` annotations.
3. Define a `PasswordEncoder` bean using `@Bean` annotation.
4. Define a `UserDetailsService` bean using `@Bean` annotation and `InMemoryUserDetailsManager`.
5. Use `User.withUsername()` instead of `withUser()` to create a UserDetails object.
6. Use `authorizeRequests()` and `formLogin()` methods in a `SecurityFilterChain` bean to configure HTTP security.
7. Remove deprecated and unused APIs like `withUser()` and `withDefaultPasswordEncoder()`.
8. Update the code to use the latest Spring Security APIs and configurations.

Hint:
```yaml
description: |
  Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
  Instead, define a `UserDetailsService` and `PasswordEncoder` bean.
  Use a `SecurityFilterChain` bean to configure HTTP security instead of extending `WebSecurityConfigurerAdapter`.
message: |
  Consider defining a `UserDetailsService` and `PasswordEncoder` bean.
  Use a `SecurityFilterChain` bean for HTTP security configurations.
```

### Evaluation

| Criteria                                                                 | Result         |
|--------------------------------------------------------------------------|----------------|
| Removed deprecated `configure(AuthenticationManagerBuilder)`            |  Yes           |
| Replaced with `UserDetailsService` and `PasswordEncoder` bean           |  Yes |
| Removed `WebSecurityConfigurerAdapter`                                   |  Yes           |
| The output compiles                                                      | <todo>          |
| Preserves behavior (in-memory auth, user with role)                      | Yes        |